In [ ]:
# import cv2
# import mediapipe as mp

In [ ]:
# mp_hand = mp.solutions.hands#Landmarks
# mp_draw = mp.solutions.drawing_utils#drawing landmarks

In [ ]:
# data = mp_hand.Hands(1)

In [1]:
import cv2

In [2]:
import cv2
import mediapipe as mp
import pyautogui
import mss
import mss.tools
import ctypes
import time

# --- STEP 1: FIX DPI ACCURACY (For High-Res Screens) ---
try:
    ctypes.windll.shcore.SetProcessDpiAwareness(1)
except Exception:
    ctypes.windll.user32.SetProcessDPIAware()

# --- STEP 2: SETUP ---
screen_w, screen_h = pyautogui.size()
sct = mss.mss()
# Get primary monitor info for coordinate offsets
main_monitor = sct.monitors[1]

video = cv2.VideoCapture(0)
mp_hand = mp.solutions.hands
data = mp_hand.Hands(max_num_hands=1, min_detection_confidence=0.8)

# Smoothing Variables
p_t_x, p_t_y, p_i_x, p_i_y = 0, 0, 0, 0
smooth_factor = 0.2

print("System Ready. Press 'S' to capture the area between your fingers.")

while True:
    success, img = video.read()
    if not success: break
    
    img = cv2.flip(img, 1)
    h, w, c = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = data.process(img_rgb)

    if result.multi_hand_landmarks:
        for hand_lms in result.multi_hand_landmarks:
            # 1. Get Landmarks
            thumb = hand_lms.landmark[4]
            index = hand_lms.landmark[8]
            
            # 2. Apply Smoothing Logic
            p_t_x = p_t_x + (thumb.x * screen_w - p_t_x) * smooth_factor
            p_t_y = p_t_y + (thumb.y * screen_h - p_t_y) * smooth_factor
            p_i_x = p_i_x + (index.x * screen_w - p_i_x) * smooth_factor
            p_i_y = p_i_y + (index.y * screen_h - p_i_y) * smooth_factor

            # 3. Calculate Box Coordinates (Desktop Scale)
            x1, y1 = int(min(p_t_x, p_i_x)), int(min(p_t_y, p_i_y))
            x2, y2 = int(max(p_t_x, p_i_x)), int(max(p_t_y, p_i_y))
            box_w, box_h = x2 - x1, y2 - y1

            # 4. Calculate Box Coordinates (Webcam Preview Scale)
            cam_x1, cam_y1 = int((x1/screen_w)*w), int((y1/screen_h)*h)
            cam_x2, cam_y2 = int((x2/screen_w)*w), int((y2/screen_h)*h)

            # 5. DRAW BOX
            cv2.rectangle(img, (cam_x1, cam_y1), (cam_x2, cam_y2), (0, 255, 0), 2)

            # 6. TRIGGER: CAPTURE
            if cv2.waitKey(1) & 0xFF == ord('s'):
                if box_w > 20 and box_h > 20:
                    # Define capture region relative to Monitor 1
                    region = {
                        "top": main_monitor["top"] + y1,
                        "left": main_monitor["left"] + x1,
                        "width": box_w,
                        "height": box_h
                    }
                    
                    sct_img = sct.grab(region)
                    mss.tools.to_png(sct_img.rgb, sct_img.size, output="snip.png")
                    print(f"Captured Snip! Coordinates: {x1, y1} Size: {box_w}x{box_h}")
                    
                    # Flash Effect
                    cv2.rectangle(img, (cam_x1, cam_y1), (cam_x2, cam_y2), (255, 255, 255), -1)

    cv2.imshow("AI Sniper - Press 'S' to Capture", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

System Ready. Press 'S' to capture the area between your fingers.
